In [17]:
#!pip install gensim
#!pip install pyLDAvis
#!pip install numpy
#!pip install wordcloud
#!pip install matplotlib
!pip install seaborn

import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns
import pyLDAvis.gensim_models
import numpy as np
import pickle
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nhgirma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
df = pd.read_csv('clean_processed_tweet_data.csv')

In [19]:
df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,screen_name,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,2022-08-07 22:31:20+00:00,"<a href=""http://twitter.com/download/android"" ...",\n\nLets focus in one very specific zone of th...,0.000000e+00,0.08125,en,4.0,2.0,i_ameztoy,20497,20497,NaN,City,i_ameztoy,NaN
1,2022-08-07 22:31:16+00:00,"<a href=""http://twitter.com/download/android"" ...",#China's media explains the military reasons ...,-1.000000e-01,0.10000,en,691.0,201.0,ZIisq,65,65,NaN,China,ZIisq,NaN
2,2022-08-07 22:31:07+00:00,"<a href=""http://twitter.com/download/android"" ...","China even cut off communication, they don't a...",0.000000e+00,0.00000,en,0.0,0.0,Fin21Free,85,85,NaN,XiJinping,Fin21Free,Netherlands
3,2022-08-07 22:31:06+00:00,"<a href=""http://twitter.com/download/android"" ...","Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.35000,en,0.0,0.0,Fin21Free,85,85,NaN,XiJinping,Fin21Free,Netherlands
4,2022-08-07 22:31:04+00:00,"<a href=""http://twitter.com/download/iphone"" r...","I’m sorry, I thought Taiwan was an independen...",-6.938894e-18,0.55625,en,1521.0,381.0,VizziniDolores,910,910,NaN,NaN,VizziniDolores,"Ayent, Schweiz"


In [20]:
import re
def clean_text_cleaner(clean_text: str) -> str:
    cleaned_text = re.sub('\n', '', clean_text)
    cleaned_text = re.findall(r'[a-zA-Z]+', cleaned_text)
    cleaned_text = " ".join(cleaned_text)
    cleaned_text = re.sub(r'http.*', "", cleaned_text)
    cleaned_text = re.sub("^RT.*:", "", cleaned_text)

    return cleaned_text

In [21]:
df['original_text'] = df['original_text'].apply(clean_text_cleaner)
df['original_text']

0        Lets focus in one very specific zone of the we...
1        China s media explains the military reasons fo...
2        China even cut off communication they don t an...
3        Putin to XiJinping I told you my friend Taiwan...
4        I m sorry I thought Taiwan was an independent ...
                               ...                        
21995    A good infographic of China s missile launches...
21996    A good infographic of China s missile launches...
21997                     Reuters Thanks Pelosi smart move
21998    Taiwan people s Desire for Unification With Ch...
21999    Taiwan people s Desire for Unification With Ch...
Name: original_text, Length: 22000, dtype: object

In [23]:
df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,screen_name,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,2022-08-07 22:31:20+00:00,"<a href=""http://twitter.com/download/android"" ...",Lets focus in one very specific zone of the we...,0.000000e+00,0.08125,en,4.0,2.0,i_ameztoy,20497,20497,NaN,City,i_ameztoy,NaN
1,2022-08-07 22:31:16+00:00,"<a href=""http://twitter.com/download/android"" ...",China s media explains the military reasons fo...,-1.000000e-01,0.10000,en,691.0,201.0,ZIisq,65,65,NaN,China,ZIisq,NaN
2,2022-08-07 22:31:07+00:00,"<a href=""http://twitter.com/download/android"" ...",China even cut off communication they don t an...,0.000000e+00,0.00000,en,0.0,0.0,Fin21Free,85,85,NaN,XiJinping,Fin21Free,Netherlands
3,2022-08-07 22:31:06+00:00,"<a href=""http://twitter.com/download/android"" ...",Putin to XiJinping I told you my friend Taiwan...,1.000000e-01,0.35000,en,0.0,0.0,Fin21Free,85,85,NaN,XiJinping,Fin21Free,Netherlands
4,2022-08-07 22:31:04+00:00,"<a href=""http://twitter.com/download/iphone"" r...",I m sorry I thought Taiwan was an independent ...,-6.938894e-18,0.55625,en,1521.0,381.0,VizziniDolores,910,910,NaN,NaN,VizziniDolores,"Ayent, Schweiz"


In [10]:
print("The number of missing value(s): {}".format(df.isnull().sum().sum()))
print("Columons having columns value:{}".format(df.columns[df.isnull().any()]))

The number of missing value(s): 42367
Columons having columns value:Index(['polarity', 'favorite_count', 'retweet_count', 'possibly_sensitive',
       'hashtags', 'place'],
      dtype='object')


In [24]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stop = stopwords.words('english')
stemmer = PorterStemmer()
# convert all original_text fields to strings
df['original_text'] = df['original_text'].astype(str)
df['clean_text'] = df['original_text'].str.replace('[^\w\s]','')
df['clean_text'] = df['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['clean_text'] = df['clean_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['clean_text'] = df['clean_text'].str.replace('\w*\d\w*','')

df['clean_text'] = df['clean_text'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))




<>:7: DeprecationWarning: invalid escape sequence \w
<>:10: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \w
<>:10: DeprecationWarning: invalid escape sequence \w
/tmp/ipykernel_265667/1958113020.py:7: DeprecationWarning: invalid escape sequence \w
  df['clean_text'] = df['original_text'].str.replace('[^\w\s]','')
/tmp/ipykernel_265667/1958113020.py:10: DeprecationWarning: invalid escape sequence \w
  df['clean_text'] = df['clean_text'].str.replace('\w*\d\w*','')
/tmp/ipykernel_265667/1958113020.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_text'] = df['original_text'].str.replace('[^\w\s]','')
/tmp/ipykernel_265667/1958113020.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_text'] = df['clean_text'].str.replace('\w*\d\w*','')


In [25]:
df['clean_text'][0:3]

0    let focu one specif zone western coast gt long...
1    china media explain militari reason area drill...
2    china even cut commun anwer phonecal us but cl...
Name: clean_text, dtype: object

In [26]:
df.to_pickle('./trained_model/cleaned_tweet.pkl')

In [30]:
object_file =open('./trained_model/cleaned_tweet.pkl', 'rb')
object_file= pickle.load('trained_model/cleaned_tweet.pkl')


TypeError: file must have 'read' and 'readline' attributes